# **Import libraries**

In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from fuzzywuzzy import fuzz

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# **Upload data**

In [ ]:
train1=pd.read_csv('/content/train_subset_1.csv')
train2=pd.read_csv('/content/train_subset_2.csv')
train3=pd.read_csv('/content/train_subset_3.csv')
train4=pd.read_csv('/content/train_subset_4.csv')
test=pd.read_csv('/content/test.csv')

let's discover the four subsets

In [ ]:
pd.set_option("display.max_columns", None)
train1.head()

,unnamed:0,personDescription,min.YearlyIncome,customerOrder,grossWeight,netWeight,isRecyclable?,additionalFeaturesInMarket,promotionName,storeKind,storeSales,storeCost,groceryArea,frozenArea,meatArea,cost,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,review_score
0,mc_ID_0,"Single Female with four children, education: b...",$10K+,"Cleaning Supplies from Household department, O...",28.1997,-26.6008,recyclable,"['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,deluxe,8.76 Millions,4.2924 Millions,2037.64,481.98,323,602.7575,FLORIANOPOLIS,5C,amount: -27.591715047463765,-48.506655980581606,navegantes,sC,-26.85907968190116,-48.63953562951828,5.0
1,mc_ID_1,Single Female with three children and educatio...,€42K+,"Snack Foods from Snack Foods department, Order...",16.571,14.972,non recyclable,NaN,Budget Bargains,Supermarket,$6.36 Millions,1.9716 Millions,2049.72,457.36,NaN,708.665,abaete,MG,-19.16050583709177,-45.45377571289521,betim,MG,-19.95866259017359,-44.19878943272789,4.0
2,mc_ID_2,"MaRrIeD MaLe wItH TwO ChIlDrEn,eDuCaTiOn: HiGh...",30K+,"Magazines from Periodicals department, Ordered...",-28.6358,data: 27.1822,recyclable,['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,1322.21,523.32,348.85,val: 564.2647,Sao PauLo,Sp,data: -23.50609758451153,-46.73241237719997,PRAIE GRANDE,$P,-24.026786758814687,-46.49304889148245,5.0
3,mc_ID_3,"Single Female with three children, education: ...",30K+ annually,"Dairy from Dairy department, Ordered Brand : C...",14.2161,amount: 11.2944,recyclable,['Coffee Bar'],Savings Galore,NaN,1.98 Millions,0.6732 Millions,NaN,440.92,293.95,724.5119,sao paulo,SP,-23.665780780743223,-46.6821276135092,Sao Paulo,SP,-23.69289565500035,-46.68021046531817,1.0
4,mc_ID_4,"Married Female with five children , education:...",50000,"Vegetables from Produce department, Ordered Br...",12.6172,9.71,non recyclable,"['Bar For Salad', 'Ready Food', 'Coffee Bar', ...",Sale Winners,Deluxe,"11.56 ,000,000illions",4.9708 Millions,1872.19,593.93,395.95,519.7574,capivari,SP,-23.0122137105142,-47.52203973160603,sao paulo,\tSP,val: -23.539824031911863,-46.434679443937874,5.0


In [ ]:
train1.tail()

,unnamed:0,personDescription,min.YearlyIncome,customerOrder,grossWeight,netWeight,isRecyclable?,additionalFeaturesInMarket,promotionName,storeKind,storeSales,storeCost,groceryArea,frozenArea,meatArea,cost,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,review_score
10390,mc_ID_10390,"Married Female with four children, education: ...",30K+,"Eggs from Eggs department, Ordered Brand : Giant",13.155,10.2478,non recyclable,NaN,Save It (Sale),NaN,$5.13 Millions,2.565 Millions,2047.28,457.36,306.05,467.4051,sao pau1o,$P,-23.715914526390176,-46.7819351646447,tupa,SP,-21.93532101918528,-50.49756182976886,1.0
10391,mc_ID_10391,"Married Male with three kids, education: parti...",10k,"Electrical from Household department, Ordered ...",31.3976,29.944,non recyclable,"['Video Store', 'Florist', 'Ready Food', 'Bar ...",Fantastic Discounts,Gourmet,6.51 Millions,2.7993 Millions,1426.33,465.54,310.3,718.9158,feira de sañtana,ba,-12.225319704968523,-38.94806634596223,MaUa,SP,-23.656676689970634,-46.44420755410021,4.0
10392,mc_ID_10392,"Married Female with five children, education: ...",30K+,"Hot Beverages from Beverages department, Order...",25.0018,22.0946,non recyclable,['Florist'],Save It (Sale),Supermark3t,3.36 Millions,1.4784 Millions,1320.15,NaN,348.85,427.7617,CAMPINAS,SP,-22.92991227115403,data: -47.07189552530681,PIRACICABA,$P,-22.71683928585045,-47.65736585405934,5.0
10393,mc_ID_10393,"Married Female with three children, education:...",10000,"Electrical from Household department, Ordered ...",-24.275,19.9142,recyclable,"['Bar For Salad', 'Coffee Bar', 'Ready Food', ...",Two Day Sale,gOURMeT,"4.92 ,000,000illions",1.968 Millions,1424.85,465.54,310.3,663.5824,sao paulo,SP,-23.61662227782904,-46.515106571469765,ribeir4o preto,5P,-21.17121748495008,-47.81941847157649,3.0
10394,mc_ID_10394,"Married Female with two kidren, education: bac...",50K+,Beer and Wine from Alcoholic Beverages departm...,13.0969,10.1897,non recyclable,['Florist'],NaN,NaN,1.62 Millions,0.6966 Millions,unknown,523.32,349.1,279.1251,livramento de nossa senhora,BA,-13.649606510639636,-41.84169289659037,sao paulo,\tSP,-23.627894217323533,-46.642414903687474,5.0


In [ ]:
train1.shape

(10395, 25)

In [ ]:
train2.head()

,field_unnamed:_0,field_person_description,min__yearly_income,customer_order,is_recyclable?,additional_features_in_market,field_promotion_name,store_kind,store_sales,store_cost,grocery_area,store_area,frozen_area,meat_area,cost,field_customer_city,customer_state,customer_lat,customer_lng,field_seller_city,seller_state,seller_lat,seller_lng,review_score,weights
0,mc_ID_10395,"Married Male with four children, education: pa...",10K+,Carbonated Beverages from Beverages department...,recyclable,"['Bar For Salad', 'Coffee Bar', 'Ready Food', ...",Not Specified,Deluxe,5.24Ms,2.4628 Millions,NaN,2572.86,503.07,334.51,290.7357,sao paulu,Sp\n,-23.56910958090484,-46.57235346003806,m@ringa,PR,val: -23.42880633856904,-51.97183135000928,5.0,"{'gross': val: 10.5822, 'net': 7.675}"
1,mc_ID_10396,"Single Male with four children, education: bac...",70K+,"Canned Shrimp from Canned Foods department, Or...",non recyclable,"['Ready Food', 'Bar For Salad', 'Coffee Bar', ...",Shelf Clearing Days,Gourmet,6.22 Millions,2.2392 Millions,1424.85,2200.59,465.54,310.3,533.2508,cachoeiras de macacu,RJ,-22.46501185656128,-42.657960353794586,belo horizonte,MG,-19.921126734952896,-43.94119674937962,1.0,"{'gross': 20.2049, 'net': data: 18.606}"
2,mc_ID_10397,"Married Female with three children, education:...",Income: 130K+,"Vegetables from Frozen Foods department, Order...",recyclable,"['Bar For Salad', 'Florist', 'Coffee Bar', 'Re...",Promo Big,Deluxe,4.38 Millions,1.8396 Millions,2057.14,3145.5,654.13,436.09,267.3576,teresina,PI,-5.055889969866429,-42.76615847343458,5anto andre,SP,-23.68645557285476,-46.52061165889549,1.0,"{'gross': val: 17.2978, 'net': 14.376}"
3,mc_ID_10398,"Single Male with four children, education: hig...",$110K+,"Frozen Desserts from Frozen Foods department, ...",recyclable,"['Coffee Bar', 'Florist', 'Ready Food', 'Video...",Lottery Cash Registerion,Deluxe,€2.4M,1.2255 Millions,1735.48,2575.08,503.07,336.07,313.0678,bras1lia,df_,-15.84372601762118,-48.11611031164195,joinvill3,SC,amount: -26.270134671231133,-48.80876106346335,amount: 5.0,"{'gross': 14.2597, 'net': 12.8062}"
4,mc_ID_10399,"Single Female with one child, education: parti...",10K+,"Specialty from Produce department, Ordered Bra...",recyclable,['Florist'],NaN,Supermarket,10.17 Millions,3.2544 Millions,1322.92,2189.1,523.32,348.85,689.3663,DRACENA,_SP,-21.4988715,-51.53786099999999,sao paulo,$P,-23.59798552039933,-46.55547278711988,val: 5.0,"{'gross': 25.2925, 'net': 22.24}"


In [ ]:
train2.shape

(10395, 25)

In [ ]:
train3.head()

,z153yi8,u5erb551yo,vbi5ustafos,v1sves1l,nj8d1js6mly,zw9u23wfoat,bslnykoygnx,ixet0uzuuoh,bcqe6tdkash,gnyebdyjsq,laedo3e,u3onbs8utn,910alza9eq,qvwwq8qe00,ykri3z1g,gds8lo,zy5q4ch85,snbu4rj,y02zr8iy,oep16bl,hvh8c4,fqs6141,vrhnlr48,v0zfxgo3xh8,47scottr,98nwv2,m8b70rwh
0,mc_ID_1350,"Married Female with four children, education: ...",30K+,"Dairy from Dairy department, Ordered Brand : G...",21.0771,18.17 kg,2.907200000000003,no,"['Ready Food', 'Bar For Salad', 'Coffee Bar', ...",Roller Savings (High),gourmet,2.67 M,0.801 M,1424.85,2200.69,465.54,310.26,499.7265,cardoso,SP,amount: -20.07337070860132,-49.91260766329853,bRasIlIA,DF,-15.656754584921352,data: -47.808582687413306,val: 4.0
1,mc_ID_1351,"Single Male with three children, education: pa...",30K+,"Fruit from Produce department, Ordered Brand :...",30.3801,27.3275,3.052599999999998,yes,['Florist'],You Save Days,supermarke7,5.7 M,2.223 M,NaN,2194.54,523.32,348.85,689.3663,_sao paulo,sp_,-23.666837080690684,-46.66437239509124,OSaSCO,SP,-23.552456794881493,-46.78012559033992,5.0
2,mc_ID_1352,"Married_Female_w/_three_children,_education:_B...",50K+,"Meat from Deli department, Ordered Brand : Cut...",val: nan,21.2225,NaN,no,['Coffee Bar'],NaN,Supermarket,5.97 M,1.9104 M,1238.65,1972.94,440.92,NaN,364.1649,Porto Alegre,RS,-30.02159907383571,-51.178723476398005,Ribeirao Preto,SP,-21.152969235356927,amount: -47.77227625506376,val: 5.0
3,mc_ID_1353,Married Female with one child and education: b...,50K or more,"Dairy from Dairy department, Ordered Brand : B...",31.543,28.6358,2.9072,no,"['Florist', 'Ready Food', 'Coffee Bar', 'Video...",Save Price,Deluxe,$9.66 M,3.0912 M,1736.78,2573.89,503.07,333.47,419.9167,lagoa da prata,MG,data: -20.022568491315106,-45.54469897672237,petropolis,Rj,-22.549033360244323,-43.210094724044055,5.0
4,mc_ID_1354,Married Female with four children / education:...,$10K+,"Fruit from Produce department, Ordered Brand :...",9.8699,6.9482,measure: 2.9216999999999995,no,"['Bar For Salad', 'Coffee Bar', 'Ready Food', ...",Sale : Double Down,Deluxe,15.76 M,6.9344 M,NaN,3145.5,NaN,436.09,500.7202,RIACHINHO,MG,amount: -16.223266212215982,-45.997340336760026,ASSIS,SP,-22.6796532539352,val: -50.44287891853987,5.0


In [ ]:
train3.shape

(10395, 27)

In [ ]:
train4.head()

,Unnamed: 0,personDescription,min._yearly_income_info,data_customer_order,grossWeight,netWeight,is_recyclable?_field,additional_features_in_market_measure,PN,SK,store_sales,SC,GA,Store Area,field_frozen_area,field_meat_area,metric_cost,customer_city,CUS56,customer_lat,var_customer_lng,field_seller_city,seller_state,seller_lat_metric,seller_lng,REV54
0,mc_ID_11745,Single Male with three children + education: b...,50 Thousand,Bathroom Products from Health and Hygiene depa...,19.0421,14.6813,no,"['Ready Food', 'Bar For Salad', 'Florist', 'Co...",Roller Savings (High),Gourmet,4.47 M,1.3857 M,1566.86,2211.99,385.45,257.63,637.6939,caMpo ERE,sC,-26.39359270318612,-53.08265733215715,maringa,pr,-23.437975269077903,-51.91385959802448,1.0
1,mc_ID_11746,"Married Male with two children, education...",30K+,"Breakfast Foods from Frozen Foods department, ...",NaN,8.1837,no,"['Video Store', 'Bar For Salad', 'Ready Food',...",Bag Stuffer,Deluxa,$1.71 M,0.7524 M,1734.5,2575.43,503.07,335.38,662.2226,cURITIBa,PR,-25.48129914876794,-49.27064708076061,Belo Horizonte,SP,-19.81011863229108,-43.98472732006267,4.0
2,mc_ID_11747,"Married Female with one child, education: part...",10K+,"Vegetables from Frozen Foods department, Order...",21.2225,NaN,yes,['Coffee Bar'],Discount Frenzy,NaN,8.88 M,4.3512 M,1237.2,1970.94,440.92,293.95,val: 760.4943,SAO PAULO,$P,-23.74981810254469,-46.69876375197105,BRUSQUE,SC,-27.14105704919435,-48.96146112936535,3.0
3,mc_ID_11748,"Married Female with three children, education:...",10 Thousand,"Bread from Baked Goods department, Ordered Bra...",10.9746,8.0674,yes,[],Super Wallet Savers,Supermarket,9.44 M,4.1536 M,1505.26,NaN,227.8,151.9,733.1937,sao paulo,SP,-23.59573427739748,-46.59839507900305,ARARAS\n,_sp,-22.36330214293956,-47.38673630322003,measure: 5.0
4,mc_ID_11749,"Single Male with five children, education: par...",10K+,"Dairy from Dairy department, Ordered Brand : G...",NaN,14.6813,yes,[],Price Winners,Supermarket,17.95 M,7.3595 M,1623.48,2195.29,NaN,226.24,586.7537,belo horizonte,mG,amount: -19.91690663641869,-44.00258750903103,sao paulo,sp,-23.59798552039933,-46.55547278711988,3.0


In [ ]:
train4.shape

(10397, 26)

In [ ]:
test.head()

,ID,Person Description,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,review_score
0,0,"Single Male with four children, education: par...","Candy from Snacks department, Ordered Brand : ...","['Florist', 'Ready Food', 'Video Store', 'Bar ...",Fantastic Discounts,Gourmet,11.76 Millions,4.704 Millions,-31.8337,28.7811,3.05 kg,yes,50K+,2201.06,1424.85,465.54,308.73,florianopol1s,sc,-27.591715047463765,-48.506655980581606,navegantes,sç,-26.85907968190116,-48.63953562951828,5.0
1,1,"Married Female with three children, education:...","Vegetables from Frozen Foods department, Order...","['Florist', 'Coffee Bar', 'Ready Food', 'Bar F...",Bag Stuffer,Deluxe,2.16 Millions,0.6696 Millions,29.944,27.0368,2.9072,no,70K+,2577.16,1735.17,505.07,336.59,abaete,MG,measure: -19.16050583709177,-45.45377571289521,betim,mg,-19.95866259017359,data: -44.19878943272789,4.0
2,2,"sINGLE fEMALE WITH THREE CHILDREN, EDUCATION: ...","Dairy from Dairy department, Ordered Brand : Club","['Bar For Salad', 'Video Store', 'Ready Food',...",Pick Your Savings,Deluxe,€1.6M,0.8235 Millions,29.2172,26.31,2.9072,no,130K+,2837.58,2038.11,481.98,321.26,sao paulo,sP,-23.50609758451153,-46.73241237719997,PrAIa grandE,SP,-24.026786758814687,measure: -46.49304889148245,5.0
3,3,"Single Male with three children, education: pa...","Frozen Desserts from Frozen Foods department, ...","['Coffee Bar', 'Video Store', 'Bar For Salad',...",Price Winners,_Deluxe,$8.82 Millions,4.41 Millions,28.0543,val: 25.0018,3.0525,no,10K+,2859.04,1871.16,593.93,394.58,sao paulo,\tSP,-23.665780780743223,-46.6821276135092,sao paulo,SP,-23.69289565500035,-46.68021046531817,1.0
4,4,"Single Male w/ two children, education: High S...","Hot Beverages from Beverages department, Order...",['Florist'],Dollar Days,Supermarket,"4.32 ,000,000illions",1.9872 Millions,23.5482,-20.641,2.9072,no,30K+,2193.97,1320.15,523.32,348.85,capivari,SP,data: -23.0122137105142,measure: -47.52203973160603,sao paulo,SP,-23.539824031911863,-46.434679443937874,5.0


In [ ]:
test.shape

(19942, 26)

As we noticed the 4 subsets does not have same no of columns and their column names are different ,so we should standarize them to one naming and format


**Handling train1**

In [ ]:
train1.columns

Index(['unnamed:0', 'personDescription', 'min.YearlyIncome', 'customerOrder',
       'grossWeight', 'netWeight', 'isRecyclable?',
       'additionalFeaturesInMarket', 'promotionName', 'storeKind',
       'storeSales', 'storeCost', 'groceryArea', 'frozenArea', 'meatArea',
       'cost', 'customer_city', 'customer_state', 'customer_lat',
       'customer_lng', 'seller_city', 'seller_state', 'seller_lat',
       'seller_lng', 'review_score'],
      dtype='object')

In train1 subset ,i will

Rename columns so they all have the same standard name

put columns that are in other subsets ,but not in this subset with null

In [ ]:
train1= train1.rename(columns={
    'personDescription': 'Person Description',
    'customerOrder': 'Customer Order',
    'additionalFeaturesInMarket': 'Additional Features in market',
    'promotionName': 'Promotion Name',
    'storeKind': 'Store Kind',
    'storeSales': 'Store Sales',
    'storeCost': 'Store Cost',
    'grossWeight': 'Gross Weight',
    'netWeight': 'Net Weight',
    'isRecyclable?': 'Is Recyclable?',
    'min.YearlyIncome': 'Yearly Income',
    'groceryArea': 'Grocery Area',
    'frozenArea': 'Frozen Area',
    'meatArea': 'Meat Area',
    'customer_city': 'Customer City',
    'customer_state': 'Customer State',
    'customer_lat': 'Customer Latitude',
    'customer_lng': 'Customer Longitude',
    'seller_city': 'Seller City',
    'seller_state': 'Seller State',
    'seller_lat': 'Seller Latitude',
    'seller_lng': 'Seller Longitude',
    'review_score': 'Review Score',
    'cost': 'Cost'
})


train1['Store Area'] = np.nan
train1['Package Weight'] = np.nan
train1.drop(columns=['unnamed:0'], inplace=True)

In [ ]:
train1.head()

,Person Description,Yearly Income,Customer Order,Gross Weight,Net Weight,Is Recyclable?,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Grocery Area,Frozen Area,Meat Area,Cost,Customer City,Customer State,Customer Latitude,Customer Longitude,Seller City,Seller State,Seller Latitude,Seller Longitude,Review Score,Store Area,Package Weight
0,"Single Female with four children, education: b...",$10K+,"Cleaning Supplies from Household department, O...",28.1997,-26.6008,recyclable,"['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,deluxe,8.76 Millions,4.2924 Millions,2037.64,481.98,323,602.7575,FLORIANOPOLIS,5C,amount: -27.591715047463765,-48.506655980581606,navegantes,sC,-26.85907968190116,-48.63953562951828,5.0,NaN,NaN
1,Single Female with three children and educatio...,€42K+,"Snack Foods from Snack Foods department, Order...",16.571,14.972,non recyclable,NaN,Budget Bargains,Supermarket,$6.36 Millions,1.9716 Millions,2049.72,457.36,NaN,708.665,abaete,MG,-19.16050583709177,-45.45377571289521,betim,MG,-19.95866259017359,-44.19878943272789,4.0,NaN,NaN
2,"MaRrIeD MaLe wItH TwO ChIlDrEn,eDuCaTiOn: HiGh...",30K+,"Magazines from Periodicals department, Ordered...",-28.6358,data: 27.1822,recyclable,['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,1322.21,523.32,348.85,val: 564.2647,Sao PauLo,Sp,data: -23.50609758451153,-46.73241237719997,PRAIE GRANDE,$P,-24.026786758814687,-46.49304889148245,5.0,NaN,NaN
3,"Single Female with three children, education: ...",30K+ annually,"Dairy from Dairy department, Ordered Brand : C...",14.2161,amount: 11.2944,recyclable,['Coffee Bar'],Savings Galore,NaN,1.98 Millions,0.6732 Millions,NaN,440.92,293.95,724.5119,sao paulo,SP,-23.665780780743223,-46.6821276135092,Sao Paulo,SP,-23.69289565500035,-46.68021046531817,1.0,NaN,NaN
4,"Married Female with five children , education:...",50000,"Vegetables from Produce department, Ordered Br...",12.6172,9.71,non recyclable,"['Bar For Salad', 'Ready Food', 'Coffee Bar', ...",Sale Winners,Deluxe,"11.56 ,000,000illions",4.9708 Millions,1872.19,593.93,395.95,519.7574,capivari,SP,-23.0122137105142,-47.52203973160603,sao paulo,\tSP,val: -23.539824031911863,-46.434679443937874,5.0,NaN,NaN


In [ ]:
train1.shape

(10395, 26)

**Handling train2**

In [ ]:
train2.columns

Index(['field_unnamed:_0', 'field_person_description', 'min__yearly_income',
       'customer_order', 'is_recyclable?', 'additional_features_in_market',
       'field_promotion_name', 'store_kind', 'store_sales', 'store_cost',
       'grocery_area', 'store_area', 'frozen_area', 'meat_area', 'cost',
       'field_customer_city', 'customer_state', 'customer_lat', 'customer_lng',
       'field_seller_city', 'seller_state', 'seller_lat', 'seller_lng',
       'review_score', 'weights'],
      dtype='object')

In [ ]:
train2['weights'].head(20)

,weights
0,"{'gross': val: 10.5822, 'net': 7.675}"
1,"{'gross': 20.2049, 'net': data: 18.606}"
2,"{'gross': val: 17.2978, 'net': 14.376}"
3,"{'gross': 14.2597, 'net': 12.8062}"
4,"{'gross': 25.2925, 'net': 22.24}"
5,"{'gross': 11.4979, 'net': 10.0298}"
6,"{'gross': -23.4028, 'net': 20.4957}"
7,"{'gross': 12.3555, 'net': 10.9019}"
8,"{'gross': 27.6183, 'net': 26.1647}"
9,"{'gross': 26.6008, 'net': measure: 23.5482}"


In train2 subset i will:

 extract net weight and gross weight from weights column

 put columns that are not in other subsets with null

rename columns,so all subsets have the same standard


In [ ]:


gross_list = []
net_list = []

for w in train2['weights']:
    w_str = str(w)

    gross_match = re.search(r"gross.*?(-?\d+\.?\d*)", w_str)
    gross_list.append(float(gross_match.group(1)) if gross_match else None)

    net_match = re.search(r"net.*?(-?\d+\.?\d*)", w_str)
    net_list.append(float(net_match.group(1)) if net_match else None)


train2['Gross Weight'] = gross_list
train2['Net Weight'] = net_list
#i will let package weight to null untill i handle the 2 cols of gross&net weight of the full data set
train2['Package Weight'] =np.nan
train2.drop(columns=['weights','field_unnamed:_0' ], inplace=True)
train2= train2.rename(columns={
    'field_person_description': 'Person Description',
    'customer_order': 'Customer Order',
    'additional_features_in_market': 'Additional Features in market',
    'field_promotion_name': 'Promotion Name',
    'store_kind': 'Store Kind',
    'store_sales': 'Store Sales',
    'store_cost': 'Store Cost',
    'is_recyclable?': 'Is Recyclable?',
    'min__yearly_income': 'Yearly Income',
    'grocery_area': 'Grocery Area',
    'store_area': 'Store Area',
    'frozen_area': 'Frozen Area',
    'meat_area': 'Meat Area',
    'field_customer_city': 'Customer City',
    'customer_state': 'Customer State',
    'customer_lat': 'Customer Latitude',
    'customer_lng': 'Customer Longitude',
    'field_seller_city': 'Seller City',
    'seller_state': 'Seller State',
    'seller_lat': 'Seller Latitude',
    'seller_lng': 'Seller Longitude',
    'review_score': 'Review Score',
    'cost': 'Cost'
})

In [ ]:
train2.head()

,Person Description,Yearly Income,Customer Order,Is Recyclable?,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Grocery Area,Store Area,Frozen Area,Meat Area,Cost,Customer City,Customer State,Customer Latitude,Customer Longitude,Seller City,Seller State,Seller Latitude,Seller Longitude,Review Score,Gross Weight,Net Weight,Package Weight
0,"Married Male with four children, education: pa...",10K+,Carbonated Beverages from Beverages department...,recyclable,"['Bar For Salad', 'Coffee Bar', 'Ready Food', ...",Not Specified,Deluxe,5.24Ms,2.4628 Millions,NaN,2572.86,503.07,334.51,290.7357,sao paulu,Sp\n,-23.56910958090484,-46.57235346003806,m@ringa,PR,val: -23.42880633856904,-51.97183135000928,5.0,10.5822,7.6750,NaN
1,"Single Male with four children, education: bac...",70K+,"Canned Shrimp from Canned Foods department, Or...",non recyclable,"['Ready Food', 'Bar For Salad', 'Coffee Bar', ...",Shelf Clearing Days,Gourmet,6.22 Millions,2.2392 Millions,1424.85,2200.59,465.54,310.3,533.2508,cachoeiras de macacu,RJ,-22.46501185656128,-42.657960353794586,belo horizonte,MG,-19.921126734952896,-43.94119674937962,1.0,20.2049,18.6060,NaN
2,"Married Female with three children, education:...",Income: 130K+,"Vegetables from Frozen Foods department, Order...",recyclable,"['Bar For Salad', 'Florist', 'Coffee Bar', 'Re...",Promo Big,Deluxe,4.38 Millions,1.8396 Millions,2057.14,3145.5,654.13,436.09,267.3576,teresina,PI,-5.055889969866429,-42.76615847343458,5anto andre,SP,-23.68645557285476,-46.52061165889549,1.0,17.2978,14.3760,NaN
3,"Single Male with four children, education: hig...",$110K+,"Frozen Desserts from Frozen Foods department, ...",recyclable,"['Coffee Bar', 'Florist', 'Ready Food', 'Video...",Lottery Cash Registerion,Deluxe,€2.4M,1.2255 Millions,1735.48,2575.08,503.07,336.07,313.0678,bras1lia,df_,-15.84372601762118,-48.11611031164195,joinvill3,SC,amount: -26.270134671231133,-48.80876106346335,amount: 5.0,14.2597,12.8062,NaN
4,"Single Female with one child, education: parti...",10K+,"Specialty from Produce department, Ordered Bra...",recyclable,['Florist'],NaN,Supermarket,10.17 Millions,3.2544 Millions,1322.92,2189.1,523.32,348.85,689.3663,DRACENA,_SP,-21.4988715,-51.53786099999999,sao paulo,$P,-23.59798552039933,-46.55547278711988,val: 5.0,25.2925,22.2400,NaN


In [ ]:
train2.shape

(10395, 26)

**Handling train3**

In [ ]:
train3.columns

Index(['z153yi8', 'u5erb551yo', 'vbi5ustafos', 'v1sves1l', 'nj8d1js6mly',
       'zw9u23wfoat', 'bslnykoygnx', 'ixet0uzuuoh', 'bcqe6tdkash',
       'gnyebdyjsq', 'laedo3e', 'u3onbs8utn', '910alza9eq', 'qvwwq8qe00',
       'ykri3z1g', 'gds8lo', 'zy5q4ch85', 'snbu4rj', 'y02zr8iy', 'oep16bl',
       'hvh8c4', 'fqs6141', 'vrhnlr48', 'v0zfxgo3xh8', '47scottr', '98nwv2',
       'm8b70rwh'],
      dtype='object')

In [ ]:
train3.shape

(10395, 27)

In train3 subset ,I will:

Use fuzzywuzzy to detect the similarities between columns and make reference train1 subset and also manually,so i can rename them


In [ ]:
def best_column_match(df, df_ref, sample_size=50):
    mapping = {}

    for col1 in df.columns:

        values1 = df[col1].dropna().astype(str).unique()[:sample_size]

        best_col = None
        best_score = 0

        for col2 in df_ref.columns:
            values2 = df_ref[col2].dropna().astype(str).unique()[:sample_size]

            scores = []
            for v1 in values1:
                best_match = 0
                for v2 in values2:
                    score = fuzz.token_sort_ratio(v1.lower(), v2.lower())
                    if score > best_match:
                        best_match = score
                scores.append(best_match)


            avg_score = sum(scores)/len(scores) if scores else 0

            if avg_score > best_score:
                best_score = avg_score
                best_col = col2

        mapping[col1] = (best_col, round(best_score,2))

    return mapping

In [ ]:
mapping = best_column_match(train3, train1)
for k,v in mapping.items():
    print(f"{k}  -> {v[0]} ->  (score={v[1]})")

z153yi8  -> Grocery Area ->  (score=40.5)
u5erb551yo  -> Person Description ->  (score=92.94)
vbi5ustafos  -> Yearly Income ->  (score=99.4)
v1sves1l  -> Customer Order ->  (score=87.54)
nj8d1js6mly  -> Net Weight ->  (score=67.9)
zw9u23wfoat  -> Net Weight ->  (score=73.02)
bslnykoygnx  -> Seller Longitude ->  (score=49.28)
ixet0uzuuoh  -> Is Recyclable? ->  (score=87.42)
bcqe6tdkash  -> Additional Features in market ->  (score=98.0)
gnyebdyjsq  -> Promotion Name ->  (score=96.0)
laedo3e  -> Store Kind ->  (score=96.52)
u3onbs8utn  -> Store Sales ->  (score=58.0)
910alza9eq  -> Grocery Area ->  (score=55.18)
qvwwq8qe00  -> Grocery Area ->  (score=77.08)
ykri3z1g  -> Grocery Area ->  (score=63.7)
gds8lo  -> Frozen Area ->  (score=96.8)
zy5q4ch85  -> Meat Area ->  (score=78.18)
snbu4rj  -> Cost ->  (score=66.38)
y02zr8iy  -> Customer City ->  (score=64.98)
oep16bl  -> Customer State ->  (score=90.48)
hvh8c4  -> Seller Latitude ->  (score=54.62)
fqs6141  -> Seller Longitude ->  (score=54

In [ ]:
# u5erb551yo  Person Description
# vbi5ustafos Yearly Income
# v1sves1l Customer Order
# nj8d1js6mly Gross Weight

#  zw9u23wfoat  Net Weight
#  bslnykoygnx Package Weight
#  ixet0uzuuoh Is Recyclable?

# bcqe6tdkash Additional Features in market
# gnyebdyjsq Promotion Name

# laedo3e Store Kind
# u3onbs8utn Store Sales

# 910alza9eq Store Cost
#  y02zr8iy Customer City
#  	oep16bl Customer State
# hvh8c4 Customer Latitude
# gds8lo Frozen Area

# fqs6141 Customer Longitude
# vrhnlr48 Seller City
# v0zfxgo3xh8 Seller State
# 47scottr Seller Latitude
# 98nwv2 Seller Longitude
# qvwwq8qe00  Grocery Area
# ykri3z1g Store Area
#   zy5q4ch85 Meat Area

#  	snbu4rj Cost
# m8b70rwh  	Review Score


In [ ]:
train3.drop(columns=['z153yi8'], inplace=True)
train3=train3.rename(columns={
    "u5erb551yo": "Person Description",
    "vbi5ustafos": "Yearly Income",
    "v1sves1l": "Customer Order",
    "nj8d1js6mly": "Gross Weight",
    "zw9u23wfoat": "Net Weight",
    "bslnykoygnx": "Package Weight",
    "ixet0uzuuoh": "Is Recyclable?",
    "bcqe6tdkash": "Additional Features in market",
    "gnyebdyjsq": "Promotion Name",
    "laedo3e": "Store Kind",
    "u3onbs8utn": "Store Sales",
    "910alza9eq": "Store Cost",
    "y02zr8iy": "Customer City",
    "oep16bl": "Customer State",
    "hvh8c4": "Customer Latitude",
    "gds8lo": "Frozen Area",
    "fqs6141": "Customer Longitude",
    "vrhnlr48": "Seller City",
    "v0zfxgo3xh8": "Seller State",
    "47scottr": "Seller Latitude",
    "98nwv2": "Seller Longitude",
    "qvwwq8qe00": "Grocery Area",
    "ykri3z1g": "Store Area",
    "zy5q4ch85": "Meat Area",
    "snbu4rj": "Cost",
    "m8b70rwh": "Review Score"
})

In [ ]:
train3.head()

,Person Description,Yearly Income,Customer Order,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Grocery Area,Store Area,Frozen Area,Meat Area,Cost,Customer City,Customer State,Customer Latitude,Customer Longitude,Seller City,Seller State,Seller Latitude,Seller Longitude,Review Score
0,"Married Female with four children, education: ...",30K+,"Dairy from Dairy department, Ordered Brand : G...",21.0771,18.17 kg,2.907200000000003,no,"['Ready Food', 'Bar For Salad', 'Coffee Bar', ...",Roller Savings (High),gourmet,2.67 M,0.801 M,1424.85,2200.69,465.54,310.26,499.7265,cardoso,SP,amount: -20.07337070860132,-49.91260766329853,bRasIlIA,DF,-15.656754584921352,data: -47.808582687413306,val: 4.0
1,"Single Male with three children, education: pa...",30K+,"Fruit from Produce department, Ordered Brand :...",30.3801,27.3275,3.052599999999998,yes,['Florist'],You Save Days,supermarke7,5.7 M,2.223 M,NaN,2194.54,523.32,348.85,689.3663,_sao paulo,sp_,-23.666837080690684,-46.66437239509124,OSaSCO,SP,-23.552456794881493,-46.78012559033992,5.0
2,"Married_Female_w/_three_children,_education:_B...",50K+,"Meat from Deli department, Ordered Brand : Cut...",val: nan,21.2225,NaN,no,['Coffee Bar'],NaN,Supermarket,5.97 M,1.9104 M,1238.65,1972.94,440.92,NaN,364.1649,Porto Alegre,RS,-30.02159907383571,-51.178723476398005,Ribeirao Preto,SP,-21.152969235356927,amount: -47.77227625506376,val: 5.0
3,Married Female with one child and education: b...,50K or more,"Dairy from Dairy department, Ordered Brand : B...",31.543,28.6358,2.9072,no,"['Florist', 'Ready Food', 'Coffee Bar', 'Video...",Save Price,Deluxe,$9.66 M,3.0912 M,1736.78,2573.89,503.07,333.47,419.9167,lagoa da prata,MG,data: -20.022568491315106,-45.54469897672237,petropolis,Rj,-22.549033360244323,-43.210094724044055,5.0
4,Married Female with four children / education:...,$10K+,"Fruit from Produce department, Ordered Brand :...",9.8699,6.9482,measure: 2.9216999999999995,no,"['Bar For Salad', 'Coffee Bar', 'Ready Food', ...",Sale : Double Down,Deluxe,15.76 M,6.9344 M,NaN,3145.5,NaN,436.09,500.7202,RIACHINHO,MG,amount: -16.223266212215982,-45.997340336760026,ASSIS,SP,-22.6796532539352,val: -50.44287891853987,5.0


In [ ]:
train3.shape

(10395, 26)

**Handling train4**

In [ ]:
train4.columns

Index(['Unnamed: 0', 'personDescription', 'min._yearly_income_info',
       'data_customer_order', 'grossWeight', 'netWeight',
       'is_recyclable?_field', 'additional_features_in_market_measure', 'PN',
       'SK', 'store_sales', 'SC', 'GA', 'Store Area', 'field_frozen_area',
       'field_meat_area', 'metric_cost', 'customer_city', 'CUS56',
       'customer_lat', 'var_customer_lng', 'field_seller_city', 'seller_state',
       'seller_lat_metric', 'seller_lng', 'REV54'],
      dtype='object')

In [ ]:
train4.shape

(10397, 26)

As in train1 subset ,i will:

Rename columns (I used the function with fuzzywuzzy To ensure more that my renaming )correct

put columns that are in other subsets ,but not in this subset with null

In [ ]:

mapping = best_column_match(train4, train1)
for k,v in mapping.items():
    print(f"{k}  -> {v[0]} ->  (score={v[1]})")

Unnamed: 0  -> Gross Weight ->  (score=42.92)
personDescription  -> Person Description ->  (score=94.42)
min._yearly_income_info  -> Yearly Income ->  (score=97.24)
data_customer_order  -> Customer Order ->  (score=88.46)
grossWeight  -> Gross Weight ->  (score=69.14)
netWeight  -> Net Weight ->  (score=71.02)
is_recyclable?_field  -> Is Recyclable? ->  (score=76.79)
additional_features_in_market_measure  -> Additional Features in market ->  (score=96.0)
PN  -> Promotion Name ->  (score=98.0)
SK  -> Store Kind ->  (score=97.9)
store_sales  -> Store Sales ->  (score=57.06)
SC  -> Net Weight ->  (score=55.7)
GA  -> Grocery Area ->  (score=82.04)
Store Area  -> Grocery Area ->  (score=64.34)
field_frozen_area  -> Frozen Area ->  (score=94.94)
field_meat_area  -> Meat Area ->  (score=75.24)
metric_cost  -> Cost ->  (score=68.54)
customer_city  -> Customer City ->  (score=65.16)
CUS56  -> Customer State ->  (score=96.62)
customer_lat  -> Customer Latitude ->  (score=56.1)
var_customer_lng  

In [ ]:
train4.drop(columns=['Unnamed: 0'], inplace=True)

train4=train4.rename(columns={

    "personDescription": "Person Description",
    "min._yearly_income_info": "Yearly Income",
    "data_customer_order": "Customer Order",
    "grossWeight": "Gross Weight",
    "netWeight": "Net Weight",
    "is_recyclable?_field": "Is Recyclable?",
    "additional_features_in_market_measure": "Additional Features in market",
    "PN": "Promotion Name",
    "SK": "Store Kind",
    "store_sales": "Store Sales",
    "SC": "Store Cost",
    "GA": "Grocery Area",
    "Store Area": "Store Area",
    "field_frozen_area": "Frozen Area",
    "field_meat_area": "Meat Area",
    "metric_cost": "Cost",
    "customer_city": "Customer City",
    "CUS56": "Customer State",
    "customer_lat": "Customer Latitude",
    "var_customer_lng": "Customer Longitude",
    "field_seller_city": "Seller City",
    "seller_state": "Seller State",
    "seller_lat_metric": "Seller Latitude",
    "seller_lng": "Seller Longitude",
    "REV54": "Review Score"
})

train4['Package Weight'] = np.nan

In [ ]:
train4.head()

,Person Description,Yearly Income,Customer Order,Gross Weight,Net Weight,Is Recyclable?,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Grocery Area,Store Area,Frozen Area,Meat Area,Cost,Customer City,Customer State,Customer Latitude,Customer Longitude,Seller City,Seller State,Seller Latitude,Seller Longitude,Review Score,Package Weight
0,Single Male with three children + education: b...,50 Thousand,Bathroom Products from Health and Hygiene depa...,19.0421,14.6813,no,"['Ready Food', 'Bar For Salad', 'Florist', 'Co...",Roller Savings (High),Gourmet,4.47 M,1.3857 M,1566.86,2211.99,385.45,257.63,637.6939,caMpo ERE,sC,-26.39359270318612,-53.08265733215715,maringa,pr,-23.437975269077903,-51.91385959802448,1.0,NaN
1,"Married Male with two children, education...",30K+,"Breakfast Foods from Frozen Foods department, ...",NaN,8.1837,no,"['Video Store', 'Bar For Salad', 'Ready Food',...",Bag Stuffer,Deluxa,$1.71 M,0.7524 M,1734.5,2575.43,503.07,335.38,662.2226,cURITIBa,PR,-25.48129914876794,-49.27064708076061,Belo Horizonte,SP,-19.81011863229108,-43.98472732006267,4.0,NaN
2,"Married Female with one child, education: part...",10K+,"Vegetables from Frozen Foods department, Order...",21.2225,NaN,yes,['Coffee Bar'],Discount Frenzy,NaN,8.88 M,4.3512 M,1237.2,1970.94,440.92,293.95,val: 760.4943,SAO PAULO,$P,-23.74981810254469,-46.69876375197105,BRUSQUE,SC,-27.14105704919435,-48.96146112936535,3.0,NaN
3,"Married Female with three children, education:...",10 Thousand,"Bread from Baked Goods department, Ordered Bra...",10.9746,8.0674,yes,[],Super Wallet Savers,Supermarket,9.44 M,4.1536 M,1505.26,NaN,227.8,151.9,733.1937,sao paulo,SP,-23.59573427739748,-46.59839507900305,ARARAS\n,_sp,-22.36330214293956,-47.38673630322003,measure: 5.0,NaN
4,"Single Male with five children, education: par...",10K+,"Dairy from Dairy department, Ordered Brand : G...",NaN,14.6813,yes,[],Price Winners,Supermarket,17.95 M,7.3595 M,1623.48,2195.29,NaN,226.24,586.7537,belo horizonte,mG,amount: -19.91690663641869,-44.00258750903103,sao paulo,sp,-23.59798552039933,-46.55547278711988,3.0,NaN


In [ ]:
train4.shape

(10397, 26)

Test

In [ ]:
test.head()

,ID,Person Description,Customer Order,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Gross Weight,Net Weight,Package Weight,Is Recyclable?,Yearly Income,Store Area,Grocery Area,Frozen Area,Meat Area,customer_city,customer_state,customer_lat,customer_lng,seller_city,seller_state,seller_lat,seller_lng,review_score
0,0,"Single Male with four children, education: par...","Candy from Snacks department, Ordered Brand : ...","['Florist', 'Ready Food', 'Video Store', 'Bar ...",Fantastic Discounts,Gourmet,11.76 Millions,4.704 Millions,-31.8337,28.7811,3.05 kg,yes,50K+,2201.06,1424.85,465.54,308.73,florianopol1s,sc,-27.591715047463765,-48.506655980581606,navegantes,sç,-26.85907968190116,-48.63953562951828,5.0
1,1,"Married Female with three children, education:...","Vegetables from Frozen Foods department, Order...","['Florist', 'Coffee Bar', 'Ready Food', 'Bar F...",Bag Stuffer,Deluxe,2.16 Millions,0.6696 Millions,29.944,27.0368,2.9072,no,70K+,2577.16,1735.17,505.07,336.59,abaete,MG,measure: -19.16050583709177,-45.45377571289521,betim,mg,-19.95866259017359,data: -44.19878943272789,4.0
2,2,"sINGLE fEMALE WITH THREE CHILDREN, EDUCATION: ...","Dairy from Dairy department, Ordered Brand : Club","['Bar For Salad', 'Video Store', 'Ready Food',...",Pick Your Savings,Deluxe,€1.6M,0.8235 Millions,29.2172,26.31,2.9072,no,130K+,2837.58,2038.11,481.98,321.26,sao paulo,sP,-23.50609758451153,-46.73241237719997,PrAIa grandE,SP,-24.026786758814687,measure: -46.49304889148245,5.0
3,3,"Single Male with three children, education: pa...","Frozen Desserts from Frozen Foods department, ...","['Coffee Bar', 'Video Store', 'Bar For Salad',...",Price Winners,_Deluxe,$8.82 Millions,4.41 Millions,28.0543,val: 25.0018,3.0525,no,10K+,2859.04,1871.16,593.93,394.58,sao paulo,\tSP,-23.665780780743223,-46.6821276135092,sao paulo,SP,-23.69289565500035,-46.68021046531817,1.0
4,4,"Single Male w/ two children, education: High S...","Hot Beverages from Beverages department, Order...",['Florist'],Dollar Days,Supermarket,"4.32 ,000,000illions",1.9872 Millions,23.5482,-20.641,2.9072,no,30K+,2193.97,1320.15,523.32,348.85,capivari,SP,data: -23.0122137105142,measure: -47.52203973160603,sao paulo,SP,-23.539824031911863,-46.434679443937874,5.0


In [ ]:
test.columns

Index(['ID', 'Person Description', 'Customer Order',
       'Additional Features in market', 'Promotion Name', 'Store Kind',
       'Store Sales', 'Store Cost', 'Gross Weight', 'Net Weight',
       'Package Weight', 'Is Recyclable?', 'Yearly Income', 'Store Area',
       'Grocery Area', 'Frozen Area', 'Meat Area', 'customer_city',
       'customer_state', 'customer_lat', 'customer_lng', 'seller_city',
       'seller_state', 'seller_lat', 'seller_lng', 'review_score'],
      dtype='object')

In [ ]:
test= test.rename(columns={
    "customer_city": "Customer City",
    "customer_state": "Customer State",
    "customer_lat": "Customer Latitude",
    "customer_lng": "Customer Longitude",
    "seller_city": "Seller City",
    "seller_state": "Seller State",
    "seller_lat": "Seller Latitude",
    "seller_lng": "Seller Longitude",
    "review_score": "Review Score"
})


Now we can concat the 4 subsets in train data

In [ ]:
train = pd.concat([train1, train2, train3, train4], axis=0, ignore_index=True)


In [ ]:
train.head()

,Person Description,Yearly Income,Customer Order,Gross Weight,Net Weight,Is Recyclable?,Additional Features in market,Promotion Name,Store Kind,Store Sales,Store Cost,Grocery Area,Frozen Area,Meat Area,Cost,Customer City,Customer State,Customer Latitude,Customer Longitude,Seller City,Seller State,Seller Latitude,Seller Longitude,Review Score,Store Area,Package Weight
0,"Single Female with four children, education: b...",$10K+,"Cleaning Supplies from Household department, O...",28.1997,-26.6008,recyclable,"['Video Store', 'Florist', 'Ready Food', 'Coff...",Dimes Off,deluxe,8.76 Millions,4.2924 Millions,2037.64,481.98,323,602.7575,FLORIANOPOLIS,5C,amount: -27.591715047463765,-48.506655980581606,navegantes,sC,-26.85907968190116,-48.63953562951828,5.0,NaN,NaN
1,Single Female with three children and educatio...,€42K+,"Snack Foods from Snack Foods department, Order...",16.571,14.972,non recyclable,NaN,Budget Bargains,Supermarket,$6.36 Millions,1.9716 Millions,2049.72,457.36,NaN,708.665,abaete,MG,-19.16050583709177,-45.45377571289521,betim,MG,-19.95866259017359,-44.19878943272789,4.0,NaN,NaN
2,"MaRrIeD MaLe wItH TwO ChIlDrEn,eDuCaTiOn: HiGh...",30K+,"Magazines from Periodicals department, Ordered...",-28.6358,data: 27.1822,recyclable,['Florist'],Shelf Emptiers,Supermarket,10.86 Millions,4.4526 Millions,1322.21,523.32,348.85,val: 564.2647,Sao PauLo,Sp,data: -23.50609758451153,-46.73241237719997,PRAIE GRANDE,$P,-24.026786758814687,-46.49304889148245,5.0,NaN,NaN
3,"Single Female with three children, education: ...",30K+ annually,"Dairy from Dairy department, Ordered Brand : C...",14.2161,amount: 11.2944,recyclable,['Coffee Bar'],Savings Galore,NaN,1.98 Millions,0.6732 Millions,NaN,440.92,293.95,724.5119,sao paulo,SP,-23.665780780743223,-46.6821276135092,Sao Paulo,SP,-23.69289565500035,-46.68021046531817,1.0,NaN,NaN
4,"Married Female with five children , education:...",50000,"Vegetables from Produce department, Ordered Br...",12.6172,9.71,non recyclable,"['Bar For Salad', 'Ready Food', 'Coffee Bar', ...",Sale Winners,Deluxe,"11.56 ,000,000illions",4.9708 Millions,1872.19,593.93,395.95,519.7574,capivari,SP,-23.0122137105142,-47.52203973160603,sao paulo,\tSP,val: -23.539824031911863,-46.434679443937874,5.0,NaN,NaN


In [ ]:
train.shape

(41582, 26)

In [ ]:
train.to_csv('train_datathon_fin.csv', index=False)
test.to_csv('test_datathon_fin.csv', index=False)